In [ ]:
using JuMP,Gurobi

In [ ]:
using CSV
data = CSV.read("data_transfered.csv")

In [ ]:
I = [1:241;]
J = [1:241;]
de = 110.25

In [ ]:
c = zeros((241,241))
for i = 1:241
    for j = 1:241
        c[i,j] = de*((data[i,8]-data[j,8])^2 + ((data[i,9]-data[j,9])*cos(data[j,8]))^2)^(1/2)
    end
end

In [ ]:
p = 10
K = [112 103 139 70 47 17 9 168 64 177]
Model_modify = Model(Gurobi.Optimizer)
@variable(Model_modify,x[i in I],Bin) # if city i is distribution center
@variable(Model_modify,y[i in I,j in J],Bin) #if city i is served by city j
@variable(Model_modify,C,Bin) #if there's more than 2 distributio centers in the first 20 cities
@objective(Model_modify,Min,sum(y[i,j]*c[i,j]*data[i,10] for i in I , j in J))
@constraints(Model_modify,begin
        [i in I], sum(y[i,j] for j in J) == 1
        [i in I , j in J], y[i,j]-x[j] <= 0
        sum(x[i] for i in I) == p
        # modification 1
        sum(x[i] for i in 1:20) <= 21*sum(x[i] for i in 30:40)
        # modification 2
        sum(x[i] for i in 1:20) >= 3C
        sum(x[i] for i in 1:20) <= 2+18C
        sum(x[i] for i in 30:40) >= C
        sum(x[i] for i in 30:40) <= 10C
        # modification 3
        x[39] + x[230] <= 1
        # modification 4
        x[39] + x[230] + x[139] + x[164] == 1
        # modification 5
        sum(x[i] for i in K) <= 3
    end)